<img src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/cian_main_theme.png" height="1000" width="1000"> 

<img align="center" src="https://www.cian.ru/promo/adv/assets/images/logo.svg" height="300" width="300"> 


# <center> Грабёж, разбой и другие незаконные деяния с помощью Python. <br> <br> Продолжаем ограбление. Задействуем союзника - Google!  </center>
---------

## 1. Что такое API?

API (Application Programming Interface) это уже готовый код, который можно всунуть в свой код!  Многие разработчики, в том числе Google и Вконтакте представляют свои уже готовые решения для вашей разработки. 

Обычно обращение к API происходит точно также как к обычной ссылке через пакет `requests`.

In [116]:
import pandas as pd
import numpy as np

import requests

Обратиться через этот пакет обычно нужно к простой банальной ссылке, в которую будет вшито то, что вы хотели бы для себя достать. Итак, заходим в [документацию по API для Google maps](https://developers.google.com/maps/documentation/webservices/), которая была заботливо для нас написана и начинаем её использовать.



Мы хотим получить расстояние, которое нужно пройти пешком или проехать на машине от каждой квартиры до ближайшего метро. Для этого мы будем использовать метод, который называется *distancematrix*. 

*Google Distance Matrix API* - это сервис, который предоставляет расстояние и время пути между указанными точками. Ссылка для запроса к API будет иметь следующую структуру

> https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&origins=Washington,DC&destinations=New+York+City,NY&key=YOUR_API_KEY

В первой строке указано название сервиса, к API которого мы обращаемся, метод который мы используем ,*distancematrix*, формат, в котором мы хотели бы получить ответ, *json*, и объекты, между которыми мы ищем расстояние, *Origins* - начало отсчета, *Destinations* - конец отсчета. Кроме того через амперсанды в ссылку можно добавить кучу других параметров. Например: *Mode* - способ перемещения, *Language* - язык, на котором мы хотели бы получить ответ на наш запрос.

Пробуем!

In [117]:
mainpage = "http://maps.googleapis.com/maps/api/distancematrix/json?"

place_one = "м. Юго-Западная, Москва"
place_two = "ул. Академика Анохина 20, Москва"
sposob = "walking"
YOU_API_KEY = ''

parameters = 'origins='+place_one+'&destinations='+place_two+'&mode='+sposob+'&language=ru-Ru'

itog_url = mainpage + parameters
itog_url

'http://maps.googleapis.com/maps/api/distancematrix/json?origins=м. Юго-Западная, Москва&destinations=ул. Академика Анохина 20, Москва&mode=walking&language=ru-Ru'

In [118]:
response = requests.get(itog_url)
response

<Response [200]>

Ура! Благосоавенный 200-всё-очень-хорошо-ответ! Попробуем сделать как мы делали раньше и посмотреть html нашей странички.

In [119]:
content = response.content
content

b'{\n   "destination_addresses" : [\n      "\xd1\x83\xd0\xbb. \xd0\x90\xd0\xbd\xd0\xbe\xd1\x85\xd0\xb8\xd0\xbd\xd0\xb0 \xd0\x90\xd0\xba\xd0\xb0\xd0\xb4\xd0\xb5\xd0\xbc\xd0\xb8\xd0\xba\xd0\xb0, 20, \xd0\x9c\xd0\xbe\xd1\x81\xd0\xba\xd0\xb2\xd0\xb0, \xd0\xa0\xd0\xbe\xd1\x81\xd1\x81\xd0\xb8\xd1\x8f, 119602"\n   ],\n   "origin_addresses" : [ "\xd0\xae\xd0\xb3\xd0\xbe-\xd0\x97\xd0\xb0\xd0\xbf\xd0\xb0\xd0\xb4\xd0\xbd\xd0\xb0\xd1\x8f, \xd0\x9c\xd0\xbe\xd1\x81\xd0\xba\xd0\xb2\xd0\xb0, \xd0\xa0\xd0\xbe\xd1\x81\xd1\x81\xd0\xb8\xd1\x8f, 119526" ],\n   "rows" : [\n      {\n         "elements" : [\n            {\n               "distance" : {\n                  "text" : "0,9 \xd0\xba\xd0\xbc",\n                  "value" : 897\n               },\n               "duration" : {\n                  "text" : "11 \xd0\xbc\xd0\xb8\xd0\xbd.",\n                  "value" : 687\n               },\n               "status" : "OK"\n            }\n         ]\n      }\n   ],\n   "status" : "OK"\n}\n'

Ух! Какая страшная штука. Но тем не менее она похожа именно на то, что нам нужно. Как я это понял? Интуиция... Но почему тогда она такая некрасивая? Всё дело в том, что мы пытаемся применить к нашему контенту те же самые методы, что мы использовали для html разметки. Тем не менее данные были скачены в формате json. Если бы мы указали в ссылке для скачки lxml-формат, то мы могли бы даже поработать с нашими данными красивым супом. 

Тем не менее, мы указали другой формат, потому что он более удобный. Смотрите!

In [120]:
content = response.json()
content

{'destination_addresses': ['ул. Анохина Академика, 20, Москва, Россия, 119602'],
 'origin_addresses': ['Юго-Западная, Москва, Россия, 119526'],
 'rows': [{'elements': [{'distance': {'text': '0,9 км', 'value': 897},
     'duration': {'text': '11 мин.', 'value': 687},
     'status': 'OK'}]}],
 'status': 'OK'}

Секундочку... Это что? Словарь? Да! Это словарь! 

In [121]:
type(content)

dict

Вытаскиваем из этого словаря всё, что нам необходимо. 

In [122]:
content['destination_addresses']

['ул. Анохина Академика, 20, Москва, Россия, 119602']

In [123]:
content['rows'][0]['elements'][0]['duration']['text']

'11 мин.'

Точно также можно качать расстояние между объектами по координатам! Подгружаем наш ЦИАНовский датасет!

In [124]:
df = pd.read_csv('clean_data_for_work.csv', header=0, sep=',' ,index_col=0)
longtitude = list(df['longitude'])
latitude = list(df['latitude'])
metro = list(df['metro_name'])

In [125]:
def infa_downloader(place_one,place_two,sposob):
    
    # создлали ссылку по входным параметрам
    mainpage = "http://maps.googleapis.com/maps/api/distancematrix/json?"
    parameters = 'origins='+place_one+'&destinations='+place_two+'&mode='+sposob+'&language=ru-Ru'+'&key='
    itog_url = mainpage + parameters
    
    # сделали запрос и вытащили инфу
    response = requests.get(itog_url)
    content = response.json()
    return(content)

In [126]:
str(longtitude[0])+','+str(latitude[0])

'37.544467,55.860792'

In [127]:
metro[0]

'Петровско-Разумовская,'

In [128]:
infa_downloader(str(latitude[1001])+','+str(longtitude[1001]),metro[1001],"walking")

{'destination_addresses': ['Бунинская аллея, Москва, Россия, 117041'],
 'origin_addresses': ['ул. 3-я Барышевская, Щербинка, Московская обл., Россия, 142171'],
 'rows': [{'elements': [{'distance': {'text': '7,0 км', 'value': 7006},
     'duration': {'text': '1 час. 26 мин.', 'value': 5140},
     'status': 'OK'}]}],
 'status': 'OK'}

In [129]:
infa_downloader(str(latitude[1001])+','+str(longtitude[1001]),metro[1001],"driving")

{'destination_addresses': ['Бунинская аллея, Москва, Россия, 117041'],
 'origin_addresses': ['ул. 3-я Барышевская, Щербинка, Московская обл., Россия, 142171'],
 'rows': [{'elements': [{'distance': {'text': '17,1 км', 'value': 17056},
     'duration': {'text': '35 мин.', 'value': 2111},
     'status': 'OK'}]}],
 'status': 'OK'}

Вроде бы всё хорошо. Дело осталось за малым - написать цикл! 

# НЕТ! 

Ничего подобного. Сначала нужно изучить ограничения.

<img src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/Workspace%201_180.png" height="700" width="700">

Неожиданно выясняется, что мы можем делать в течение суток не более 2500 запросов. Чёрт возьми! Какая боль, какая обида и какие страдания... Ограничения выяснили, попробуем теперь написать цикл. Авось прокатит...


In [ ]:
walk_dist  = [ ]
drive_dist = [ ]

for i in range(len(metro)):
    walk_dist.append( vremia_finder(str(latitude[i])+','+str(longtitude[i]),metro[i],"walking") )
    drive_dist.append( vremia_finder(str(latitude[i])+','+str(longtitude[i]),metro[i],"driving") )

Неожиданно вспоминаем, что такой цикл не заработает, потому что метро есть не везде... Переписываем цикл...

In [58]:
walk_dist  = [ ]
drive_dist = [ ]

for i in range(len(metro)):
    try:
        walk_dist.append( vremia_finder(str(latitude[i])+','+str(longtitude[i]),metro[i],"walking") )
    except Exception:
        walk_dist.append(np.nan)
    
    try:
        drive_dist.append( vremia_finder(str(latitude[i])+','+str(longtitude[i]),metro[i],"driving") )
    except Exception:
        drive_dist.append(np.nan)    
        
    # Мы хотим видеть где именно находится наш цикл!
    if i%10 == 0:
        print(i,drive_dist[i],walk_dist[i])

0 10 мин. 51 мин.
10 32 мин. 7 ч. 45 мин.
20 30 мин. 42 мин.
30 6 мин. 12 мин.
40 31 мин. 4 ч. 47 мин.
50 31 мин. 4 ч. 47 мин.
60 7 мин. 10 мин.
70 17 мин. 1 час. 31 мин.
80 nan nan
90 9 мин. 28 мин.
100 30 мин. 3 ч. 48 мин.
110 24 мин. 3 ч. 15 мин.
120 5 мин. 12 мин.
130 31 мин. 3 ч. 51 мин.
140 19 мин. 1 час. 7 мин.
150 17 мин. 1 час. 31 мин.
160 33 мин. 5 ч. 13 мин.
170 7 мин. 9 мин.
180 23 мин. 1 час. 26 мин.
190 14 мин. 1 час. 6 мин.
200 14 мин. 1 час. 6 мин.
210 22 мин. 1 час. 36 мин.
220 nan nan
230 9 мин. 28 мин.
240 37 мин. 4 ч. 46 мин.
250 nan nan
260 14 мин. 1 час. 14 мин.
270 5 мин. 19 мин.
280 6 мин. 19 мин.
290 17 мин. 1 час. 29 мин.
300 23 мин. 1 час. 25 мин.
310 7 мин. 18 мин.
320 14 мин. 1 час. 14 мин.
330 19 ч. 31 мин. 11 дн. 13 ч.
340 25 мин. 3 ч. 40 мин.
350 17 мин. 1 час. 31 мин.
360 nan nan
370 22 мин. 2 ч. 52 мин.
380 10 мин. 57 мин.
390 16 мин. 1 час. 18 мин.
400 nan nan
410 19 мин. 1 час. 7 мин.
420 31 мин. 4 ч. 47 мин.
430 19 мин. 1 час. 7 мин.
440 2 мин. 7 ми

KeyboardInterrupt: 

Цикл работает довольно быстро и обрубается как только число отведённых нам запросов заканчивается.

``` 
{'destination_addresses': [],
 'error_message': 'You have exceeded your daily request quota for this API. We recommend registering for a key at the Google Developers Console: https://console.developers.google.com/apis/credentials?project=_',
 'origin_addresses': [],
 'rows': [],
 'status': 'OVER_QUERY_LIMIT'}

```


Что нам с этим делать? 

* Выход первый: качать несколько дней, качать с разных компов
* Выход второй: обмануть гугл, подключив кусок кода, который динамически меняет наш IP

Но об этом чуть позже...  Когда мы станем совсем продвинутыми юзерами...